# Registering and Managing ONNX in Seeq

This notebook serves as a practical guide for working with ONNX using the `SeeqONNXClient`. It includes README sections with details about the client, its exposed functions, and how to use them effectively. Through step-by-step examples, you’ll learn how to:

- Register ONNX models with Seeq and scoping them to Workbooks.
- Update registered ONNX.
- List registered ONNX.
- Delete registered ONNX no longer needed.

> **Note:** ONNX Registration is a licensed feature. Please contact Seeq Support for information on licensing to enable **External ML**.

In [1]:
# If you do not have ONNX installed, install it using the following command:
!pip install onnx

# Note: After installing ONNX, please restart the kernel to ensure the changes take effect.

# ONNX Description

The function `describe_onnx` prints details about the **Inputs** and **Outputs** of an ONNX, including:

- Tensor name
- Tensor type
- Tensor shape

In [2]:
import onnx
from source.seeq_onnx_helper import describe_onnx

In [3]:
model = onnx.load("example_mlp_regressor.onnx")

describe_onnx(model)


=== Model: MLP Regressor ===

Inputs:
  - input: type=DOUBLE, shape=['?', 2]

Outputs:
  - variable: type=FLOAT, shape=['?', 1]


## Input tensor type requirement: DOUBLE

> **Note:**  
> It is preferable to **export input types as DOUBLE** when exporting models to ONNX. This ensures compatibility with Seeq that require `DOUBLE` precision for inputs. By setting input types to `DOUBLE` from the start, you can avoid potential type mismatches and unnecessary casting operations after model export.

If Seeq rejects your ONNX registration request due to a **type mismatch**, you can cast the input to `DOUBLE` to meet the requirement.

### How to cast from `FLOAT` to `DOUBLE`:

If you need to convert an input tensor from `FLOAT` to `DOUBLE`, you can perform the casting within the ONNX model using the `Cast` operator. Here’s an example of how to do this using Python and the `onnx` library:

#### Python code to cast input type

```python
import onnx
from source.seeq_onnx_helper import cast_double_to_float

model = onnx.load("model.onnx")
modified_model = cast_double_to_float(model)
onnx.save(modified_model, "modified_model.onnx")


# ONNX Registration

> ONNX Registration is a licensed feature. Ensure that you have the appropriate license before attempting to register. For more information or to enable **External ML**, please contact Seeq Support.

## ONNX Registration Clients

The `seeq_onnx_helper` provides a client for registering ONNX in Seeq.

> **Note:** When using Data Lab, if you want to use `seeq_onnx_helper` from any folder/notebook, add the following code at the beginning of your notebook:
```python
import sys
sys.path.append('/home/datalab/SPy Documentation/Machine Learning')
```

### `SeeqONNXClient`
This client uses the requests library to make direct HTTP API calls to Seeq server, useful when the Seeq SDK is not available.

``` python
from source.seeq_onnx_helper import SeeqONNXClient
from source.seeq_onnx_helper import get_auth_token

hostname = "https://tenant.seeq.com"
token = "auth_token"
seeq_onnx_client = SeeqONNXClient(hostname, token)
```

#### How to get authentication token

> When you are using `SeeqONNXClient` client, you need to pass a `auth_token` as a parameter. You can either extract it from API requests or if you have any version of `SPy` installed, you can access it from `spy.client.auth_token` after `spy.login()`.

> If you do not have `SPy`, you can use the function `get_auth_token` to generate a new authentication token. This function takes `host`, `username`, and `password` as parameters.

> If you are on Single Sign-On (SSO), and do not have `username` and `password` for Seeq (or if you do not want to enter your credentials), then you can create an `Access Key`, `password` on Seeq and use that as `username` and `password`.


```python
from source.seeq_onnx_helper import get_auth_token

hostname = "https://tenant.seeq.com"
username = "username"
password = "password"
token = get_auth_token(hostname, username, password)

## Available Functions 

`SeeqONNXClient` exposes 4 core functions:

### 1. `list`

The `list` function lists all the ONNX registered in Seeq server.

Below are the details of parameters accepted by `list` function:

<div style="display: inline-block">
    
| Parameter     | Type              | Description |
|:--------------|:------------------|:------------|
| `scope`       | `str`             | *(Optional)* The ID of the workbook to filter the ONNX by scope. If not provided, all ONNX will be listed. |
| `include_archived` | `bool`             | *(Optional)* If True, includes archived ONNX in the list. Defaults to False. |
</div>

### Example usage:

```python
seeq_onnx_client.list(scope="0F02FF3F-C351-7120-A14E-9C6B5EA3FF30")
```


### 2. `register`

The `register` function provides functionality to register an **ONNX** in Seeq server.

Below are the details of parameters accepted by `register` function:

<div style="display: inline-block">
    
| Parameter     | Type              | Description |
|:--------------|:------------------|:------------|
| `name`        | `str`             | Name to assign to the registered ONNX. |
| `description` | `str`             | A short description of the ONNX's purpose. |
| `model`       | `onnx.ModelProto` | The ONNX model to register. |
| `model_type`  | `str`             | The type of the model (`PREDICTION` or `ANOMALY`). |
| `labels`      | `List[str]`       | List of input labels/features corresponding to the ONNX's input. |
| `scope`       | `UUID`            | *(Optional)* The ID of the workbook to which this item will be scoped. |
</div>

> **Note** Ensure that the labels list passed to register function maintains the same order as the features used during model training. Any mismatch in order can result in feeding incorrect signals into the model, leading to unreliable or inaccurate predictions.

Seeq supports two types of Machine Learning Models: [Prediction model and Anomaly model](https://support.seeq.com/kb/latest/cloud/machine-learning-models).

#### PREDICTION Model options

Use these as keyword arguments when `model_type="PREDICTION"`:

<div style="display: inline-block">
    
| Option        | Type   | Description |
|:--------------|:-------|:------------|
| `valueUnits`  | `str`  | *(Optional)* Unit of the predicted output signal. |
</div>


#### ANOMALY Model options

Use these as keyword arguments when `model_type="ANOMALY"`:

<div style="display: inline-block">
    
| Option             | Type   | Description |
|:-------------------|:-------|:------------|
| `outlierValue`     | `str`  | **Required.** The name of the ONNX model’s output tensor which predicts outliers (e.g., `"outlier"`). |
| `outlierPredicate` | `str`  | *(Optional)* Expression to identify outliers based on `outlierValue`. Default is `">0"`. Allowed values: `">0"`, `"<0"`. |
| `singleValue*`     | `str`  | *(Optional)* Keys starting with this prefix define ONNX model’s output tensor that expect a **single output value**. |
<!-- | `featureValue*`    | `str`  | *(Optional)* Keys starting with this prefix define signals that expect **feature-sized output** | -->
</div>

> **Note:**
> - Seeq currently accepts only ONNX models with a single input tensor of data type **DOUBLE** and output tensors of data type **DOUBLE / FLOAT / INT64**.
> - In the case of a prediction model, the first output of the ONNX will be treated as the predicted value.

### Example usage:

```python
seeq_onnx_client.register(
    name="MyModel",
    description="An ONNX model for predictions",
    model=onnx_model,
    model_type="PREDICTION",
    labels=["label1", "label2"],
    scope="0F02FF3F-C351-7120-A14E-9C6B5EA3FF30"
)
```


### 3. `update`

The `update` function provides functionality to update an already registered **ONNX** in Seeq server.

> **Note:** If an ONNX was registered with `model_type="PREDICTION"` then it cannot be changed to `model_type="ANOMALY"`

Below are the details of parameters accepted by `update` function:

<div style="display: inline-block">
    
| Parameter     | Type              | Description |
|:--------------|:------------------|:------------|
| `id`          | `str`             | The ID of the registered ONNX to update. |
| `name`        | `str`             | *(Optional)* Name to assign to the registered ONNX. |
| `description` | `str`             | *(Optional)* A short description of the ONNX's purpose. |
| `model`       | `onnx.ModelProto` | *(Optional)* The ONNX model to register. |
| `labels`      | `List[str]`       | *(Optional)* *(Required when `model` is set)* List of input labels/features corresponding to the ONNX's input. |
| `scope`       | `UUID`            | *(Optional)* The ID of the workbook to which this item will be scoped. |
</div>

> **Note** Ensure that the labels list passed to register function maintains the same order as the features used during model training. Any mismatch in order can result in feeding incorrect signals into the model, leading to unreliable or inaccurate predictions.

Seeq supports two types of Machine Learning Models: [Prediction model and Anomaly model](https://support.seeq.com/kb/latest/cloud/machine-learning-models).

#### PREDICTION Model options

Use these as keyword arguments if the registed ONNX `model_type="PREDICTION"` :

<div style="display: inline-block">
    
| Option        | Type   | Description |
|:--------------|:-------|:------------|
| `valueUnits`  | `str`  | *(Optional)* Unit of the predicted output signal. |
</div>


#### ANOMALY Model options

Use these as keyword arguments if the registed ONNX `model_type="ANOMALY"`:

<div style="display: inline-block">
    
| Option             | Type   | Description |
|:-------------------|:-------|:------------|
| `outlierValue`     | `str`  | **Required.** The name of the ONNX model’s output tensor which predicts outliers (e.g., `"outlier"`). |
| `outlierPredicate` | `str`  | *(Optional)* Expression to identify outliers based on `outlierValue`. Default is `">0"`. Allowed values: `">0"`, `"<0"`. |
| `singleValue*`     | `str`  | *(Optional)* Keys starting with this prefix define ONNX model’s output tensor that expect a **single output value**. |
<!-- | `featureValue*`    | `str`  | *(Optional)* Keys starting with this prefix define signals that expect **feature-sized output** | -->
</div>

> **Note:** Individual fields can be updated by passing values only for the fields that need to be changed.

### Example usage:

```python
seeq_onnx_client.update(
    "0F03ACF8-0E0E-E8F0-833A-122F99ACAF26",
    name="New Name",
    description="New description for the ONNNX",
    model=new_onnx_model,
    labels=["new_label1", "new_label2"],
)
```

### 4. `archive`

The `archive` function archives the ONNX registered in Seeq server.

Below are the details of parameters accepted by `archive` function:

<div style="display: inline-block">
    
| Parameter     | Type              | Description |
|:--------------|:------------------|:------------|
| `id`          | `str`             | The ID of the registered ONNX to archive. |
| `delete`      | `bool`            | *(Optional)* If True, the ONNX will be deleted; otherwise, it will be archived. Defaults to False. |
</div>

### Example usage:

```python
seeq_onnx_client.archive("0F03AD3C-DDA4-6670-AB9D-ADE5257848E1", delete=False)
```




## Examples

In the following examples, we scope the ONNX to a workbook. A new workbook at default path is created, and the Registered ONNX are scoped to it. If you prefer to use an existing workbook, you can simply provide its workbook_id.

In [4]:
from seeq import spy

# Log into Seeq Server if you're not using Seeq Data Lab:
spy.login(url='http://localhost:34216', credentials_file='../credentials.key', force=False, quiet=True)

In [5]:
# Workbook to scope Example ONNX to

workbook_path = spy.DEFAULT_WORKBOOK_PATH
workbook_context = spy._context.WorkbookContext.from_string(session=spy.session, status=spy.Status(), workbook_arg=workbook_path, worksheet_arg=None, datasource_arg=None) 
workbook_id = workbook_context.workbook_object.id

workbook_url = f"{spy.session.public_url}/workbook/{workbook_id}"
print("Scoped workbook URL:", workbook_url)

Scoped workbook URL: http://localhost:34216/workbook/0F0609A2-7A54-ECE0-A0CB-76417CA4B452


### Examples using `SeeqONNXClient`

Start by initializing the `SeeqONNXClient` using the Seeq server's hostname and your access token

In [6]:
import datetime
import onnx
from source.seeq_onnx_helper import SeeqONNXClient

hostname = spy.client.host
token = spy.client.auth_token
seeq_onnx_client = SeeqONNXClient(hostname, token)

#### Example 1: Registering an ONNX Prediction model

In this example, we register a trained machine learning model built using an `MLPRegressor` algorithm. The model was trained on two input features - `Relative Humidity` and `Wet Bulb` - to predict the `Temperature` of `Example>>Cooling Tower 1>> Area *`. It was then converted to ONNX format and saved to a file. Below, we load the ONNX from the file and register it to Seeq as `PREDICTION` model.

In [7]:
model = onnx.load("example_mlp_regressor.onnx")

describe_onnx(model)


=== Model: MLP Regressor ===

Inputs:
  - input: type=DOUBLE, shape=['?', 2]

Outputs:
  - variable: type=FLOAT, shape=['?', 1]


In [8]:
prediction_onnx_id = seeq_onnx_client.register(
    name=f"Example temperature predictor (Example>>Cooling Tower 1>>Area *) {datetime.datetime.now().isoformat()}",
    description="This ONNX model is a Multi-layer perceptron regressor (MLPRegressor) trained to predict temperature based on 'Relative Humidity' and 'Wet Bulb' features of Example>>Cooling Tower 1>>Area *",
    model=model,
    model_type="PREDICTION",
    labels=["Relative Humidity", "Wet Bulb"],
    scope=workbook_id
)

Successfully registered ONNX 'Example temperature predictor (Example>>Cooling Tower 1>>Area *) 2025-07-14T15:36:15.376956' (ID: 0F0609A2-E594-FDC0-854A-7F57E1C462AC) (Scoped to: 0F0609A2-7A54-ECE0-A0CB-76417CA4B452)


#### Example 2: Registering an ONNX Anomaly Detection model with required options

In this example, we register a trained machine learning model built using a `One-Class Support Vector Machine` algorithm. The model was trained on three input features — `Temperature`, `Relative Humidity` and `Wet Bulb` — to detect anomalies in *Example >> Cooling Tower 1 >> Area **. After training, it was converted to ONNX format and saved to a file. Below, we load the ONNX model from the file and register it to Seeq as a model of type `ANOMALY`.

In [9]:
model = onnx.load("example_oneclass_svm_ad.onnx")

describe_onnx(model)


=== Model: OneClassSVM Anomaly Detector ===

Inputs:
  - input: type=DOUBLE, shape=['?', 3]

Outputs:
  - label: type=INT64, shape=['?', 1]
  - scores: type=FLOAT, shape=['?', 1]


In [10]:
anomaly_onnx_id = seeq_onnx_client.register(
    name=f"Example outlier detector (Example>>Cooling Tower 1>>Area *) {datetime.datetime.now().isoformat()}",
    description="ONNX model to detect anomalies based on the One-Class Support Vector Machine (OneClass SVM) algorithm on Example>>Cooling Tower 1>>Area *",
    model=model,
    model_type="ANOMALY",
    labels=['Temperature', 'Relative Humidity', 'Wet Bulb'],
    scope=workbook_id,
    outlierValue="label", # One of the outputs of the ONNX
    outlierPredicate="<0"
)

Successfully registered ONNX 'Example outlier detector (Example>>Cooling Tower 1>>Area *) 2025-07-14T15:36:16.760783' (ID: 0F0609A2-E60C-77D0-AD26-9E2D5C866C59) (Scoped to: 0F0609A2-7A54-ECE0-A0CB-76417CA4B452)


#### Example 3: Registering an ONNX Anomaly Detection model along with signal outputs

In this example, we register a trained machine learning model built using a `One-Class Support Vector Machine` algorithm. The model was trained on three input features — `Temperature`, `Relative Humidity` and `Wet Bulb` — to detect anomalies in *Example >> Cooling Tower 1 >> Area **. After training, it was converted to ONNX format and saved to a file. Below, we load the ONNX model from the file and register it to Seeq as a model of type `ANOMALY`. Additionally, we add the `scores` output of ONNX as `singleValue` to enable signal calculations on Seeq.

In [11]:
model = onnx.load("example_oneclass_svm_ad.onnx")

describe_onnx(model)


=== Model: OneClassSVM Anomaly Detector ===

Inputs:
  - input: type=DOUBLE, shape=['?', 3]

Outputs:
  - label: type=INT64, shape=['?', 1]
  - scores: type=FLOAT, shape=['?', 1]


In [12]:
anomaly_onnx_id2 = seeq_onnx_client.register(
    name=f"Example outlier detector with score (Example>>Cooling Tower 1>>Area *) {datetime.datetime.now().isoformat()}",
    description="ONNX model to detect anomalies and calculate anomaly score based on the One-Class Support Vector Machine (OneClass SVM) algorithm on Example>>Cooling Tower 1>>Area *",
    model=model,
    model_type="ANOMALY",
    labels=['Temperature', 'Relative Humidity', 'Wet Bulb'],
    scope=workbook_id,
    outlierValue="label", # One of the outputs of the ONNX
    outlierPredicate="<0",
    singleValue="scores" # One of the outputs of the ONNX
)

Successfully registered ONNX 'Example outlier detector with score (Example>>Cooling Tower 1>>Area *) 2025-07-14T15:36:17.374941' (ID: 0F0609A2-EBF1-6470-BA06-E768CBFA8BDC) (Scoped to: 0F0609A2-7A54-ECE0-A0CB-76417CA4B452)


#### Example 4: Listing Registered ONNX 

We call the `list` function to list all ONNX registered in Seeq. 

In [13]:
seeq_onnx_client.list(scope=workbook_id)

,ID,Name,Description,Type,Input Labels,Feature Count,Scoped To,Archived
0,0F0609A2-E594-FDC0-854A-7F57E1C462AC,Example temperature predictor (Example>>Coolin...,This ONNX model is a Multi-layer perceptron re...,PREDICTION,"[Relative Humidity, Wet Bulb]",2,0F0609A2-7A54-ECE0-A0CB-76417CA4B452,False
1,0F0609A2-E60C-77D0-AD26-9E2D5C866C59,Example outlier detector (Example>>Cooling Tow...,ONNX model to detect anomalies based on the On...,ANOMALY,"[Temperature, Relative Humidity, Wet Bulb]",3,0F0609A2-7A54-ECE0-A0CB-76417CA4B452,False
2,0F0609A2-EBF1-6470-BA06-E768CBFA8BDC,Example outlier detector with score (Example>>...,ONNX model to detect anomalies and calculate a...,ANOMALY,"[Temperature, Relative Humidity, Wet Bulb]",3,0F0609A2-7A54-ECE0-A0CB-76417CA4B452,False


#### Example 5: Updating the Registered ONNX

Here, we update the previously registered prediction model in `Example 1` by uploading a ONNX of ML model trained with `HistGradientBoostingRegressor` algorithm. We also update the ONNX description.

In [14]:
model = onnx.load("example_hist_gb_regressor.onnx")

describe_onnx(model)


=== Model: Hist Gradient Boosting Regressor ===

Inputs:
  - input: type=DOUBLE, shape=['?', 2]

Outputs:
  - variable: type=FLOAT, shape=['?', 1]


In [15]:
seeq_onnx_client.update(
    prediction_onnx_id,
    description="Histogram-based Gradient Boosting Regression (HistGradientBoostingRegressor) trained to predict temperature based on 'Relative Humidity' and 'Wet Bulb' features of Example>>Cooling Tower 1>>Area *",
    model=model,
    labels=["Relative Humidity", "Wet Bulb"],
)

Successfully updated ONNX (ID: 0F0609A2-E594-FDC0-854A-7F57E1C462AC)


#### Example 6: Deleting a Registered ONNX

Here we archive a registered ONNX from the Seeq server.

In [16]:
seeq_onnx_client.archive(prediction_onnx_id, delete=False)

Successfully archived ONNX (ID: 0F0609A2-E594-FDC0-854A-7F57E1C462AC)
